<img src="https://www.bmw-lebanon.com/content/dam/bmw/marketMIDEAST/common/local-content/X%20Range%20Campaign/Xrange-home.jpg">

# Introduction
When you buy a new car mostly the price was determined by the manufacturer, on the other hand when buying a used car, it's usually hard to know what a fair price is. There are a variety of features of a car like mileage, transmission, type of fuel, fuel consumption, year of manufacturing, and so on these factors influence the price. In this project, we use Machine Learning to evaluate the price of a used car with a supervised learning method. These imformaiton can help the customer to make a good decision to buy used car in fair price.

In this project we are using the dataset on BMW used car. The features available
in this dataset are price, model, year, transmission, mileage, tax, mpg, enginesize  and type of fuel.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import datetime
sns.set_style("dark")


from sklearn.linear_model import LinearRegression
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, learning_curve, RandomizedSearchCV, KFold, cross_validate, train_test_split, StratifiedShuffleSplit
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.pipeline import make_pipeline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Data Preparation

## 1.1 Load Data

In [ ]:
df = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/bmw.csv')

train, test = train_test_split(df, test_size=0.3, random_state=2020)

print('train shape: {}'.format(train.shape))
print('test shape: {}'.format(test.shape))

train.sample(n=5)

In [ ]:
train['model'].unique()

In [ ]:
# remove space before model name
train['model'] = train['model'].apply(lambda x: x.strip())
train['model'].unique()

In [ ]:
train.info()

Great! There are no missing value.

In [ ]:
train.apply(lambda x: x.nunique(), axis=0)

# 2. Exporatory Data Analysis

## Numerical features

### Check correlation

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
cmap = sns.diverging_palette(10, 240, n=9)

# Plot heat map
train_corr = train.corr()
sns.heatmap(train_corr, annot=True, fmt=".2f", 
           linewidths=2, cmap=cmap, vmin=-1, vmax=1, 
           cbar_kws={"shrink": .9}, square=True);

High correlation feature are `year` `mileage` (correlation > 0.5)
* if new car price will higher than old car that make sense. (positive correlation)
* mileage is high mean that car might wear and tear so the price will drop. (negative correlation)

miles per gallon (mpg) and tax seem not significant correlate to price.

In [ ]:
sns.pairplot(train);

In [ ]:
def plot_(df, col, y='price'):
            
    fig, ax = plt.subplots(1,2,figsize=(20,5))
    ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
    ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90) 
    fig.suptitle(str(col) + ' vs price')
    

    if df[col].dtypes == 'object':
        sns.boxplot(data=df, x=col, y= y, ax=ax[0])
        sns.countplot(df[col], ax=ax[1])
        
        
    elif len(df[col].unique()) <= 10:
        sns.swarmplot(data=df, x=col, y= y, ax=ax[0])
        sns.countplot(df[col], ax=ax[1])
        
    else:
#         sns.scatterplot(data=df, x=col, y= y, ax=ax[0])
        sns.regplot(data=df, x=col, y= y, order=2, ax=ax[0])
        sns.distplot(df[col], kde=False,fit=norm, ax=ax[1])

print('SET FUNCTION COMPLETED!')

In [ ]:
# Get numerical columns
num_col = [col for col in train.columns if train[col].dtype != 'object']
print('Numerical Columns\n'+ str(num_col))

# Get categorical columns
cat_col = [col for col in train.columns if train[col].dtype == 'object']
print('\nCategorical Columns\n'+ str(cat_col))

In [ ]:
for col in num_col:
    plot_(train, col)

Some outlier that might reduce model performance so we will drop some outlier later.

## Category features

In [ ]:
for col in cat_col:
    plot_(train, col)

* Manual has lower price than other transmission.
* Most of fuel type are Diesel and Petrol respectively and Diesel has lower price than others.

## Ourlier

In [ ]:
train = train.drop(train.price[train.price >= 100000].index)

In [ ]:
pd.crosstab(train.model,train.fuelType, margins= True).style.background_gradient(cmap='Blues')

For mpg above 400 that is only for i3 model.

In [ ]:
train[(train.mpg >= 400)]

<img src="https://www.bmw.co.th/content/dam/bmw/common/all-models/i-series/i3/2017/design/BMW-i-series-i3-design-exterior-design-update-01.jpg/_jcr_content/renditions/cq5dam.resized.img.585.low.time1535122344052.jpg">

# Feature Engineering

In [ ]:
# Create age columns from year
now = datetime.datetime.now().year
train['age'] = now - train['year']
test['age'] = now - test['year']

# Change type
train['year'] = train['year'].apply(str)
test['year'] = test['year'].apply(str)

# Drop column year
data = [train,test]
for df in data:
    df.drop(columns = ['year'], inplace=True)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,8))
sns.boxplot(x="engineSize", y="price", data=train, ax=ax[0])
sns.countplot(x="engineSize", data=train, ax=ax[1])
fig.suptitle('Before Grouping Engine Size');

Most of `engineSize` are 2.0, 3.0 and 1.5 respectively so we will group `engineSize`. 

In [ ]:
# Grouping engine size. 
data = [train,test]
for df in data:
    df['engineSize'].replace([0.0, 0.6, 1.0], 1.5, inplace=True)
    df['engineSize'].replace([1.9, 2.2, 2.5], 2.0, inplace=True)
    df['engineSize'].replace([2.8, 3.2, 3.5], 3.0, inplace=True)
    df['engineSize'].replace([4.0, 4.4, 5.0, 6.6], 4.4, inplace=True)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,8))
sns.boxplot(x="engineSize", y="price", data=train, ax=ax[0])
sns.countplot(x="engineSize", data=train, ax=ax[1])
fig.suptitle('After Grouping Engine Size');

In [ ]:
plt.figure(figsize=(20,10))

sns.scatterplot(data=train, x="age", hue='engineSize', y="price", palette="deep")

* The higher `age` of car the `price` will go down.
* At the same `age` if the larger `engineSize` the price will higher. 

In [ ]:
plt.figure(figsize=(20,10))

sns.scatterplot(data=train, x="mileage", hue='engineSize', y="price",size="age", palette="deep");

* `price` will drop when smaller `engineSize` and a higher `mileage`.

In [ ]:
# Feature Extraction
data = [train,test]
for df in data:
    df['eng_age'] = df.engineSize / df.age
    df['mpg_eng'] = df.mpg / df.engineSize
    df['mpg_age'] = df.mpg / df.age

train

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
cmap = sns.diverging_palette(10, 240, n=9)

# Plot heat map
train_corr = train.corr()
sns.heatmap(train_corr, annot=True, fmt=".2f", 
           linewidths=2, cmap=cmap, vmin=-1, vmax=1, 
           cbar_kws={"shrink": .9}, square=True);

In [ ]:
def plot(df, col):
    plt.figure(figsize=(10, 5))
    plt.hist(df[col], bins='auto')
    plt.show()
    print(col)
    print("value between {:,} and {:,}".format(df[col].min(),df[col].max()))

In [ ]:
plot(train, 'price')

In [ ]:
# Log-Transform 
train.price = np.log1p(train.price)

In [ ]:
X_train = train.drop(columns = 'price')
y_train = train.price

X_test = test.drop(columns = 'price')
y_test = test.price

In [ ]:
# Get numerical columns
num_col = [col for col in X_train.columns if X_train[col].dtype != 'object']
print('Numerical Columns\n'+ str(num_col))

# Get categorical columns
cat_col = [col for col in X_train.columns if X_train[col].dtype == 'object']
print('\nCategorical Columns\n'+ str(cat_col))

In [ ]:
# Pipeline

# Numerical preprocess missing value
numerical_transformer = make_pipeline(RobustScaler())

# Categorical preprocess missing value
categorical_transformer = make_pipeline(OneHotEncoder(handle_unknown='ignore',sparse=False))

# Preprocess Numerical and Categorical variable
preprocess = ColumnTransformer(transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)])

In [ ]:
preprocess.fit_transform(X_train)

enc_cat_col = preprocess.named_transformers_['cat']['onehotencoder'].get_feature_names()
labels = np.concatenate([num_col, enc_cat_col])
X_train_ = pd.DataFrame(preprocess.fit_transform(X_train), columns=labels)

preprocess.transform(X_test)

enc_cat_col = preprocess.named_transformers_['cat']['onehotencoder'].get_feature_names()
labels = np.concatenate([num_col, enc_cat_col])
X_test_ = pd.DataFrame(preprocess.transform(X_test), columns=labels)

X_train_

# Model

We will divide data into 5 folds i.e. K=5. Now we have 5 sets of data to train and test our model. So the model will get trained and tested 5 times, but for every iteration we will use one fold as test data and rest all as training data. Note that for every iteration, data in training and test fold changes which adds to the effectiveness of this method.

K Fold cross validation helps to generalize the machine learning model and reduce overfitting

In [ ]:
kf =KFold(n_splits=5, shuffle=True, random_state=21)


print('='*30 +'BASE MODEL' + '='*30)
lar = LinearRegression()
lar.fit(X_train_, y_train)
print('slope ： {:0.2f}'.format(lar.coef_[0]))
print('intercept : {:0.2f}'.format(lar.intercept_))
print('\n')

RMSE = -1*cross_val_score(lar, X_train_, y_train, cv=kf, scoring='neg_root_mean_squared_error')

print(f'RMSE: {RMSE.mean():.5f} ±{RMSE.std():.3f}' )

In [ ]:
models = [('LinearRegression', LinearRegression()),
          ("Ridge", Ridge()),
          ("Lasso", Lasso()),
          ("ElasticNet", ElasticNet()),
          ('RandomForestRegressor', RandomForestRegressor()),
          ('GradientBoostingRegressor', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor()),
          ("LightGBM", LGBMRegressor()),
          ("CatBoost", CatBoostRegressor(verbose=False))]

print('='*30 +'RMSE TRAIN MODEL' + '='*30)

mean_rmse = []
sd_rmse = []
model_name = []
for name, model in models:
    model.fit(X_train_, y_train)
    RMSE = -1*cross_val_score(model, X_train_, y_train, cv=kf, scoring='neg_root_mean_squared_error')
    print(f'{name:<25} {RMSE.mean():.5f} ±{RMSE.std():.3f}')
    
    mean_rmse.append(RMSE.mean())
    sd_rmse.append(RMSE.std())
    model_name.append(name)

print('AVERAGE SCORE RMSE: {:.5f}'.format(np.mean(mean_rmse)))
    
base = pd.DataFrame({'model':model_name, 'mean_rmse':mean_rmse, 'sd_rmse':sd_rmse})
sns.barplot(x="mean_rmse", y="model", data=base, orient = 'h',**{'xerr': sd_rmse})
plt.title('Baseline: Cross Validation Scores')
plt.axvline(x = np.mean(mean_rmse), color = 'firebrick', linestyle = '--');

CatBoostRegressor has lowest RMSE so we  will tune hyperparameter to improve CatBoostRegressor performance by using GridSearchCV

In [ ]:
# Created an object model_CBR
cbr = CatBoostRegressor(loss_function = "RMSE", eval_metric = "RMSE", verbose = False)

# Defined the parameters of the model which we want to pass to through GridSearchCV 
parameters = {'max_depth' : [2, 5, 10],
              'learning_rate' : [0.001, 0.01, 0.1],
              'n_estimators' : [100, 200],
              'min_child_samples' : [2, 5, 10]
             }

Rgrid_search = RandomizedSearchCV(cbr, parameters, cv=kf, scoring = 'neg_mean_absolute_error', verbose = False, n_jobs = -1, return_train_score = True)

Rgrid_search.fit(X_train_, y_train)

In [ ]:
Rgrid_search.best_estimator_

In [ ]:
RMSE = -1*cross_val_score(Rgrid_search.best_estimator_, X_train_, y_train, cv=kf, scoring='neg_root_mean_squared_error')

print('SCORE RMSE: {:.5f} ±{:.3f}'.format(RMSE.mean(), RMSE.std()))

In [ ]:
feature_imp = pd.DataFrame(Rgrid_search.best_estimator_.feature_importances_, 
                           X_train_.columns,columns = ['Feature_Importances']).sort_values(by='Feature_Importances', ascending = False)
feature_imp

In [ ]:
fig, ax = plt.subplots(figsize = (15, 10))
ax = sns.barplot(x="Feature_Importances", y=feature_imp.index, data=feature_imp)

* The `engineSize/age` of car is the higest importance and follow by the `mileage` and `mpg/engineSize`.
* The larger `engineSize` the price is higher, the older cars (higher `age`) the price are cheaper.
* The higher `mileage`, `mpg` price will drop.

In [ ]:
prediction = Rgrid_search.best_estimator_.predict(X_test_) 

y_pred = np.expm1(prediction)

pd.Series(y_pred, name='price_predict')

In [ ]:
X_test_

In [ ]:
compare = pd.concat([y_test.reset_index(drop=True),pd.Series(y_pred, name='price_predict'), X_test.reset_index(drop=True)], axis='columns')
compare.sample(10)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))

ax.scatter(y_test, y_pred, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
ax.axvline(x = 50000, color = 'firebrick', alpha = 0.1, linestyle = '--')
ax.axhline(y = 37000, color = 'firebrick', alpha = 0.1, linestyle = '--');

plt.show()

There some points that much error.

In [ ]:
index_out = []
index_out.extend(compare.loc[(compare.price >=50000) & (compare.price_predict <= 37000)].index.to_list())
index_out.extend(compare.loc[(compare.price >=40000) & (compare.price_predict <= 20000)].index.to_list())
index_out.extend(compare.loc[(compare.price >=80000) & (compare.price_predict <= 50000)].index.to_list())

compare.iloc[index_out]

The `i8` and `M4` model that have a high error so we have to deliberate when we use this model to predict price of car. 